In [17]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models

# 1. Veri Setini Yükleme
main_folder = 'C:/Users/furka/PycharmProjects/Celebrity-Face-Detection/Data/train'
labels = [label for label in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, label))]

image_paths = []
for label in labels:
    label_folder = os.path.join(main_folder, label)
    for image_name in os.listdir(label_folder):
        image_path = os.path.join(label_folder, image_name)
        image_paths.append((image_path, label))

# 2. Görüntü Boyutlarını Düzenleme
image_size = (64, 64)
images = []
image_labels = []

for path, label in image_paths:
    image = cv2.imread(path)
    if image is not None:  # Check if image is loaded properly
        image = cv2.resize(image, image_size)
        images.append(image)
        image_labels.append(label)

# Convert lists to NumPy arrays
images = np.array(images)
image_labels = np.array(image_labels)

# 3. Etiketleri Sayısal Formata Çevirme
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(image_labels)

# 4. Veri Setini Train ve Test Olarak Ayırma
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# 5. Görüntüleri Normalleştirme
X_train = X_train / 255.0
X_test = X_test / 255.0

In [18]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from datetime import datetime

# 6. CNN Modeli Oluşturma
model = models.Sequential()

# Evrişim Katmanları
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Düzleştirme ve Yoğun Katmanlar
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(labels), activation='softmax'))  # Output class number
# Modeli Eğitme
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Modeli Değerlendirme
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Geçerli zamanı alıp formatlama
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"model_{current_time}.h5"

# Modeli belirli bir dizine kaydetme
output_dir = "../Output/Model/"
os.makedirs(output_dir, exist_ok=True)
model_path = os.path.join(output_dir, model_name)
model.save(model_path)

print(f"Model saved to: {model_path}")


C:\Users\furka\.conda\envs\ANN_Final\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.4548 - loss: 1.3730 - val_accuracy: 0.8042 - val_loss: 0.7329
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8008 - loss: 0.6150 - val_accuracy: 0.8986 - val_loss: 0.3078
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9307 - loss: 0.2492 - val_accuracy: 0.9196 - val_loss: 0.2407
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9581 - loss: 0.1642 - val_accuracy: 0.9476 - val_loss: 0.1679
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9727 - loss: 0.0867 - val_accuracy: 0.9685 - val_loss: 0.1230
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9857 - loss: 0.0576 - val_accuracy: 0.9650 - val_loss: 0.1330
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9892 - loss: 0.0448 - val_accuracy: 0.9301 - val_loss: 0.1807
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9869 - loss: 0.0347 - val_accuracy: 0.9755 - v


Test accuracy: 0.9755244851112366
Model saved to: ../Output/Model/model_20240811_102039.h5


In [22]:
import cv2
import numpy as np
import tensorflow as tf
from datetime import datetime
import os

# Eğitilmiş modeli yükleme
model = tf.keras.models.load_model(model_path)

# Video dosyasının yolu
video_path = 'Data/Test_Video_Jimmy Kimmel-Jodie Foster.mp4'

# Video dosyasını açma
cap = cv2.VideoCapture(video_path)

# Video çözünürlüğünü ve FPS'yi alma
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

# Geçerli zamanı alıp formatlama
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'processed_video_{current_time}.mp4'
output_dir = '../Output/Video'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, output_filename)

# Video yazıcı (output) oluşturma
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Video oynatıcı
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    # Her bir kareyi 64x64 boyutuna getirme
    image_size = (64, 64)
    resized_frame = cv2.resize(frame, image_size)
    
    # Renk kanallarını normalize etme
    normalized_frame = resized_frame / 255.0
    
    # Modelin beklediği formatta input oluşturma
    input_frame = np.expand_dims(normalized_frame, axis=0)
    
    # Model ile tahmin yapma
    predictions = model.predict(input_frame)
    predicted_label = np.argmax(predictions[0])

    # Sonucu videoya ekleme
    label_text = f'Prediction: {predicted_label}'
    cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # İşlenen kareyi video yazıcısına kaydetme
    out.write(frame)
    
    # Her kareyi ekranda gösterme (isteğe bağlı)
    cv2.imshow('Video Processing', frame)
    
    # 'q' tuşuna basıldığında videoyu kapatma
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Tüm pencereleri kapatma ve video dosyalarını serbest bırakma
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Video işleme tamamlandı. Çıktı dosyası: {output_path}')


Video işleme tamamlandı. Çıktı dosyası: ../Output/Video\processed_video_20240811_102402.mp4


In [23]:
import cv2
import numpy as np
import tensorflow as tf
from datetime import datetime
import os

# Eğitilmiş modeli yükleme
model = tf.keras.models.load_model(model_path)

# Video dosyasının yolu
video_path = 'Data/Test_Video_Jimmy Kimmel-Jodie Foster.mp4'

# Video dosyasını açma
cap = cv2.VideoCapture(video_path)

# Video çözünürlüğünü ve FPS'yi alma
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)

# Geçerli zamanı alıp formatlama
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f'processed_video_{current_time}.mp4'
output_dir = '../Output/Video'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, output_filename)

# Video yazıcı (output) oluşturma
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Video oynatıcı
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    # Her bir kareyi 64x64 boyutuna getirme
    image_size = (64, 64)
    resized_frame = cv2.resize(frame, image_size)
    
    # Renk kanallarını normalize etme
    normalized_frame = resized_frame / 255.0
    
    # Modelin beklediği formatta input oluşturma
    input_frame = np.expand_dims(normalized_frame, axis=0)
    
    # Model ile tahmin yapma (modelin yüklenmiş olduğunu varsayıyoruz)
    predictions = model.predict(input_frame)
    predicted_label = np.argmax(predictions[0])

    # Sonucu videoya ekleme
    label_text = f'Prediction: {predicted_label}'
    cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # İşlenen kareyi video yazıcısına kaydetme
    out.write(frame)
    
    # Her kareyi ekranda gösterme
    cv2.imshow('Video Processing', frame)
    
    # 'q' tuşuna basıldığında videoyu kapatma
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Tüm pencereleri kapatma ve video dosyalarını serbest bırakma
cap.release()
out.release()
cv2.destroyAllWindows()

print(f'Video işleme tamamlandı. Çıktı dosyası: {output_path}')


Video işleme tamamlandı. Çıktı dosyası: ../Output/Video\processed_video_20240811_102451.mp4


In [26]:
import os
import cv2
import numpy as np

# 2. Orijinal etiket listesini geri yükleme (eğer sınıflar bellekte saklanmışsa)
labels = [label for label in os.listdir('/Data/CNN/train') if os.path.isdir(os.path.join('/Data/CNN/train', label))]

# 3. Videoyu açma
#C:\Users\furka\PycharmProjects\Celebrity-Face-Detection\Data\Test_Video_Jimmy Kimmel-Jodie Foster.mp4
video_path = 'C:/Users/furka/PycharmProjects/Celebrity-Face-Detection/Data/Test_Video_Jimmy Kimmel-Jodie Foster.mp4'
cap = cv2.VideoCapture(video_path)

# 4. Videoyu kare kare işleme
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 5. Görüntü boyutunu ayarlama ve normalize etme
    frame_resized = cv2.resize(frame, (64, 64))
    frame_normalized = frame_resized / 255.0
    frame_expanded = np.expand_dims(frame_normalized, axis=0)  # Modelin beklediği şekil: (1, 64, 64, 3)
    
    # 6. Tahmin yapma
    predictions = model.predict(frame_expanded)
    predicted_index = np.argmax(predictions)  # En yüksek olasılıklı sınıfı al
    predicted_label = labels[predicted_index]  # Sınıf etiketini geri döndür
    
    # 7. Tahmin edilen sınıfı ekrana yazdırma
    cv2.putText(frame, f"Prediction: {predicted_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # 8. Çerçeveyi ekranda gösterme
    cv2.imshow('Video', frame)
    
    # 'q' tuşuna basıldığında videodan çık
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 9. Kaynakları serbest bırakma
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━


KeyboardInterrupt

